In [4]:
using Revise, LazySets, ConvexBodyProximityQueries, StaticArrays, Plots, BenchmarkTools, GeometryTypes

From their README:

If you want to use your custom convex objects, you can do so by extending the support function as:
        
```julia
import ConvexBodyProximityQueries.support

function ConvexBodyProximityQueries.support(obj::MyFancyShape, dir::SVector{N}) where {N}
  # do something
  return supporting_point::SVector{N}
end
```

In [5]:
import ConvexBodyProximityQueries.support
using LazySets

function ConvexBodyProximityQueries.support(obj::LazySet{N}, dir::SVector{D, N}) where {D, N}
    return LazySets.support_vector(dir, obj) |> SVector{D, N}
end

## Polygons in vertex representation

### Large number of vertices

In [57]:
using StaticArrays

polyA = @SMatrix rand(2, 8)
polyB = @SMatrix(rand(2, 8)) .+ 1.5
dir = @SVector(rand(2)) .- 0.5

2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
 0.2107671636015256
 0.2127510247989195

In [58]:
closest_points(polyA, polyB, dir)

([0.9771853591747877, 0.6794073433790953], [2.055333073672938, 1.5225231559224144])

In [59]:
@btime closest_points($polyA, $polyB, $dir)
@btime minimum_distance($polyA, $polyB, $dir)
@btime tolerance_verification($polyA, $polyB, $dir, $1.0)
@btime collision_detection($polyA, $polyB, $dir)

  149.463 ns (0 allocations: 0 bytes)
  129.848 ns (0 allocations: 0 bytes)
  124.271 ns (0 allocations: 0 bytes)
  125.126 ns (0 allocations: 0 bytes)


false

In [60]:
A = VPolygon([polyA[:, i] for i in 1:size(polyA, 2)])
B = VPolygon([polyB[:, i] for i in 1:size(polyB, 2)])

@btime closest_points($A, $B, $dir)
@btime minimum_distance($A, $B, $dir)
@btime tolerance_verification($A, $B, $dir, $1.0)
@btime collision_detection($A, $B, $dir)

  115.894 ns (0 allocations: 0 bytes)
  93.653 ns (0 allocations: 0 bytes)
  86.800 ns (0 allocations: 0 bytes)
  85.332 ns (0 allocations: 0 bytes)


false

In [61]:
closest_points(A, B, dir)

([0.9771853591747877, 0.6794073433790953], [2.055333073672938, 1.5225231559224144])

In [81]:
Ah = tohrep(A)
Ah = HPolygon([HalfSpace(SVector{2, Float64}(c.a), c.b) for c in constraints_list(Ah)])

Bh = tohrep(B)
Bh = HPolygon([HalfSpace(SVector{2, Float64}(c.a), c.b) for c in constraints_list(Bh)])

HPolygon{Float64}(HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[HalfSpace{Float64,SArray{Tuple{2},Float64,1,2}}([0.5762311284312949, 0.00027556546356155565], 1.3814567698287792), HalfSpace{Float64,SArray{Tuple{2},Float64,1,2}}([-0.05095169360997476, 0.23561745420702884], 0.4564529157618334), HalfSpace{Float64,SArray{Tuple{2},Float64,1,2}}([-0.2136632693893432, 0.43081915307125773], 0.5742572095106495), HalfSpace{Float64,SArray{Tuple{2},Float64,1,2}}([-0.668302635987674, -0.3255434671437947], -1.8692319779539677), HalfSpace{Float64,SArray{Tuple{2},Float64,1,2}}([0.35668647055569713, -0.34116870559805346], 0.21367224551567943)])

In [92]:
subtypes(LazySet, true)

39-element Array{Type,1}:
 AffineMap              
 Ball1                  
 Ball2                  
 BallInf                
 Ballp                  
 Bloating               
 CachedMinkowskiSumArray
 CartesianProduct       
 CartesianProductArray  
 ConvexHull             
 ConvexHullArray        
 Ellipsoid              
 EmptySet               
 ⋮                      
 LinearMap              
 MinkowskiSum           
 MinkowskiSumArray      
 ResetMap               
 Singleton              
 SymmetricIntervalHull  
 Translation            
 Universe               
 VPolygon               
 VPolytope              
 ZeroSet                
 Zonotope               

In [91]:
@btime σ($dir, $Ah, linear_search=true)

  366.406 ns (10 allocations: 384 bytes)


2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
 0.8506791949047251
 0.8197542793966013

In [90]:
@btime σ($dir, $Ah, linear_search=false)

  493.387 ns (12 allocations: 448 bytes)


2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
 0.8506791949047251
 0.8197542793966013

In [82]:
@btime closest_points($Ah, $Bh, $dir)
@btime minimum_distance($Ah, $Bh, $dir)
@btime tolerance_verification($Ah, $Bh, $dir, $1.0)
@btime collision_detection($Ah, $Bh, $dir)

  6.716 μs (135 allocations: 5.69 KiB)
  6.268 μs (127 allocations: 5.33 KiB)
  6.197 μs (126 allocations: 5.30 KiB)
  6.196 μs (125 allocations: 5.28 KiB)


false

### Large number of vertices

In [16]:
polyA = @SMatrix rand(2, 100)
polyB = @SMatrix(rand(2, 100)) .+ 1.5
dir = @SVector(rand(2)) .- 0.5;

In [17]:
@btime closest_points($polyA, $polyB, $dir)
@btime minimum_distance($polyA, $polyB, $dir)
@btime tolerance_verification($polyA, $polyB, $dir, $1.0)
@btime collision_detection($polyA, $polyB, $dir)

  1.933 μs (0 allocations: 0 bytes)
  1.932 μs (0 allocations: 0 bytes)
  1.426 μs (0 allocations: 0 bytes)
  1.421 μs (0 allocations: 0 bytes)


false

In [18]:
A = VPolygon([polyA[:, i] for i in 1:size(polyA, 2)])
B = VPolygon([polyB[:, i] for i in 1:size(polyB, 2)])

@btime closest_points($A, $B, $dir)
@btime minimum_distance($A, $B, $dir)
@btime tolerance_verification($A, $B, $dir, $1.0)
@btime collision_detection($A, $B, $dir)

  367.424 ns (0 allocations: 0 bytes)
  340.771 ns (0 allocations: 0 bytes)
  240.108 ns (0 allocations: 0 bytes)
  241.198 ns (0 allocations: 0 bytes)


false

### Polygon in constraint representation

In [56]:
tohrep(A)

MethodError: MethodError: no method matching tohrep(::Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}})
Closest candidates are:
  tohrep(!Matched::HPOLYGON<:AbstractHPolygon) where HPOLYGON<:AbstractHPolygon at /home/mforets/.julia/dev/LazySets/src/Interfaces/AbstractHPolygon.jl:81
  tohrep(!Matched::Union{HPolyhedron{N<:Real}, HPolytope{N<:Real}}) where N<:Real at /home/mforets/.julia/dev/LazySets/src/Sets/HPolyhedron.jl:318
  tohrep(!Matched::VPolygon{N<:Real,VN} where VN<:AbstractArray{N<:Real,1}) where N<:Real at /home/mforets/.julia/dev/LazySets/src/Sets/VPolygon.jl:170
  ...

# Using hyperrectangles

In [21]:
import ConvexBodyProximityQueries.support
using GeometryTypes: HyperSphere, HyperRectangle, HyperCube

function ConvexBodyProximityQueries.support(sphere::HyperSphere{N, T}, dir::AbstractVector) where {N, T}
    SVector{N}(sphere.center + sphere.r*normalize(dir, 2))
end

@generated function ConvexBodyProximityQueries.support(rect::HyperRectangle{N, T}, dir::AbstractVector) where {N, T}
    exprs = Array{Expr}(undef, (N,))
    for i = 1:N
        exprs[i] = :(rect.widths[$i]*(dir[$i] ≥ 0.0 ? 1.0 : -1.0)/2.0 + rect.origin[$i])
    end

    return quote
        Base.@_inline_meta
        @inbounds elements = tuple($(exprs...))
        @inbounds return SVector{N, T}(elements)
    end
end

@generated function ConvexBodyProximityQueries.support(cube::HyperCube{N, T}, dir::AbstractVector) where {N, T}
    exprs = Array{Expr}(undef, (N,))
    for i = 1:N
        exprs[i] = :(cube.width*(dir[$i] ≥ 0.0 ? 1.0 : -1.0)/2.0 + cube.origin[$i])
    end

    return quote
        Base.@_inline_meta
        @inbounds elements = tuple($(exprs...))
        @inbounds return SVector{N, T}(elements)
    end
end

In [40]:
dir = SVector{2}([1.0, 0.0])
rA = HyperRectangle(Vec(0.0, 0.0), Vec(1.0, 2.0))
rB = HyperRectangle(Vec(3.0, 0.0), Vec(1.0, 2.0))

HyperRectangle{2,Float64}([3.0, 0.0], [1.0, 2.0])

In [41]:
@btime closest_points($rA, $rB, $dir)
@btime minimum_distance($rA, $rB, $dir)
@btime tolerance_verification($rA, $rB, $dir, $1.0)
@btime collision_detection($rA, $rB, $dir)

  19.321 ns (0 allocations: 0 bytes)
  13.499 ns (0 allocations: 0 bytes)
  12.694 ns (0 allocations: 0 bytes)
  12.254 ns (0 allocations: 0 bytes)


false

In [45]:
vertices(rA)

([0.0, 0.0], [0.0, 2.0], [1.0, 0.0], [1.0, 2.0])

In [42]:
closest_points(rA, rB, dir)`

([0.5, 1.0], [2.5, 1.0])

In [54]:
minimum_distance(A, B, dir)

2.0

In [55]:
minimum_distance(rA, rB, dir)

2.0

In [43]:
oA = SVector(origin(rA))
wA = SVector(widths(rA))
A = Hyperrectangle(low=oA, high=oA+wA)

oB= SVector(origin(rB))
wB = SVector(widths(rB))
B = Hyperrectangle(low=oB, high=oB+wB)

@btime closest_points($A, $B, $dir)
@btime minimum_distance($A, $B, $dir)
@btime tolerance_verification($A, $B, $dir, $1.0)
@btime collision_detection($A, $B, $dir)

  18.587 ns (0 allocations: 0 bytes)
  12.328 ns (0 allocations: 0 bytes)
  11.734 ns (0 allocations: 0 bytes)
  15.489 ns (0 allocations: 0 bytes)


false

In [48]:
minimi

ErrorException: setindex!(::SArray{Tuple{2},Float64,1,2}, value, ::Int) is not defined.

In [52]:
A = Hyperrectangle(low=[0.0, 0.0], high=[1.0, 2.0])

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.5, 1.0], [0.5, 1.0])

In [53]:
vertices_list(A)

4-element Array{Array{Float64,1},1}:
 [1.0, 2.0]
 [0.0, 2.0]
 [1.0, 0.0]
 [0.0, 0.0]

In [44]:
closest_points(A, B, dir)

([1.0, 2.0], [3.0, 2.0])

In [ ]:
function convert(::Type{Hyperrectangle}, X::HyperRectangle{D, N}) where {D, N}
    o = origin(X)
    w = widths(X)
    return Hyperrectangle(low=o, high=o+w)
end

function convert(::Type{Hyperrectangle}, X::HyperCube{D, N}) where {D, N}
    o = origin(X)
    w = widths(X)
    return Hyperrectangle(low=o, high=o+w)
end

In [ ]:
mA = @SMatrix(rand(2, 8))
polyA = VPolygon([mA[:, i] for i in 1:8])

mB = @SMatrix(rand(2, 8)) .+ 1.5
polyB = VPolygon([mB[:, i] for i in 1:8])

In [ ]:
@btime minimum_distance($X ⊕ $Y, $M*$Z, $dir)

In [ ]:
v = rand(2)

In [ ]:
X = rand(HPolygon)
Y = rand(HPolygon)
dir = @SVector(rand(2))

In [ ]:
closest_points(X, Y, dir)

In [ ]:
minimum_distance(X, Y, dir)

In [ ]:
plot(X)
plot!(Y)

In [ ]:
@btime minimum_distance(rand(2, 2) * X, Y, dir)

In [ ]:
v = LazySets.support_vector(@SVector(rand(2)), Z1)

In [ ]:
SVector{2, Float64}(v)

In [ ]:
support(Z1, @SVector(rand(2)))

In [ ]:
@btime closest_points($Z1, $Z2, $dir)

----

In [ ]:
mA = @SMatrix(rand(2, 8))
polyA = VPolygon([mA[:, i] for i in 1:8])

mB = @SMatrix(rand(2, 8)) .+ 1.5
polyB = VPolygon([mB[:, i] for i in 1:8])

In [ ]:
dir = @SVector(rand(2)) .- 0.5

In [ ]:
# using the Library's default
@btime closest_points($mA, $mB, $dir)
@btime minimum_distance($mA, $mB, $dir)
@btime tolerance_verification($mA, $mB, $dir, $1.0)
@btime collision_detection($mA, $mB, $dir)

In [ ]:
# using the LazySets VPolygon
@btime closest_points($polyA, $polyB, $dir)
@btime minimum_distance($polyA, $polyB, $dir)
@btime tolerance_verification($polyA, $polyB, $dir, $1.0)
@btime collision_detection($polyA, $polyB, $dir)

## GeometryTypes

In [ ]:
import Pkg; Pkg.add("GeometryTypes")

In [ ]:
import ConvexBodyProximityQueries.support
using GeometryTypes: HyperSphere, HyperRectangle, HyperCube

function ConvexBodyProximityQueries.support(sphere::HyperSphere{N, T}, dir::AbstractVector) where {N, T}
    SVector{N}(sphere.center + sphere.r*normalize(dir, 2))
end

@generated function ConvexBodyProximityQueries.support(rect::HyperRectangle{N, T}, dir::AbstractVector) where {N, T}
    exprs = Array{Expr}(undef, (N,))
    for i = 1:N
        exprs[i] = :(rect.widths[$i]*(dir[$i] ≥ 0.0 ? 1.0 : -1.0)/2.0 + rect.origin[$i])
    end

    return quote
        Base.@_inline_meta
        @inbounds elements = tuple($(exprs...))
        @inbounds return SVector{N, T}(elements)
    end
end

@generated function ConvexBodyProximityQueries.support(cube::HyperCube{N, T}, dir::AbstractVector) where {N, T}
    exprs = Array{Expr}(undef, (N,))
    for i = 1:N
        exprs[i] = :(cube.width*(dir[$i] ≥ 0.0 ? 1.0 : -1.0)/2.0 + cube.origin[$i])
    end

    return quote
        Base.@_inline_meta
        @inbounds elements = tuple($(exprs...))
        @inbounds return SVector{N, T}(elements)
    end
end

In [ ]:
# using the LazySets VPolygon
@btime closest_points($polyA, $polyB, $dir)
@btime minimum_distance($polyA, $polyB, $dir)
@btime tolerance_verification($polyA, $polyB, $dir, $1.0)
@btime collision_detection($polyA, $polyB, $dir)

In [ ]:
    @testset "rectangle" begin

        @test collision_detection(rectangleA, rectangleB, vec2) == false
        @test all(ret[1] .≈ [0.5, 1.0])
        @test all(ret[2] .≈ [2.5, 1.0])

        rectangleA = HyperRectangle(Vec(0.0, 0.0), Vec(1.0, 2.0))
        rectangleB = HyperRectangle(Vec(1.0, 0.0), Vec(1.0, 2.0))
        ret = closest_points(rectangleA, rectangleB, vec2)
        @test collision_detection(rectangleA, rectangleB, vec2) == true

        rectangleA = HyperRectangle(Vec(0.0, 0.0), Vec(1.0, 2.0))
        rectangleB = HyperRectangle(Vec(0.5, 0.0), Vec(1.0, 2.0))
        ret = closest_points(rectangleA, rectangleB, vec2)
        @test collision_detection(rectangleA, rectangleB, vec2) == true
    end # rectangle

In [ ]:
using GeometryTypes

In [ ]:
HyperRectangle(Vec(0.0, 0.0), Vec(1.0, 2.0))

In [ ]:
vec2 = SVector{2}([1.0, 0.0])
rectangleA = HyperRectangle(Vec(0.0, 0.0), Vec(1.0, 2.0))
rectangleB = HyperRectangle(Vec(3.0, 0.0), Vec(1.0, 2.0))

In [ ]:
rectangleA.widths

In [ ]:
@btime ret = closest_points($rectangleA, $rectangleB, $vec2)

In [ ]:
h1 = Hyperrectangle([0.0, 0.0], [0.5, 1.0])
h2 = Hyperrectangle([3.0, 0.0], [0.5, 1.0])

In [ ]:
@btime ret = closest_points($h1, $h2, $vec2)

In [ ]:
h1 = Hyperrectangle(@SVector([0.0, 0.0]), @SVector([0.5, 1.0]))
h2 = Hyperrectangle(@SVector([3.0, 0.0]), @SVector([0.5, 1.0]))

In [ ]:
@btime ret = closest_points($h1, $h2, $vec2)

In [ ]:
GeometryTypes.Vec3f0(1.0)

## Conversions

In [ ]:
using Revise, LazySets, GeometryTypes, Test, BenchmarkTools

In [ ]:
function f(::Type{Hyperrectangle}, X::HyperRectangle{D, N}) where {D, N}
    o = origin(X)
    w = widths(X)
    return Hyperrectangle(low=o, high=o+w)
end

In [ ]:
r = HyperRectangle(Vec(0.0, 0.0), Vec(1.0, 2.0))

In [ ]:
o = origin(r)

In [ ]:
w = widths(r)

In [ ]:
Hyperrectangle(low=o, high=o+w)

In [ ]:
GeometryTypes.StaticArray{}

In [ ]:
vcat(1)

In [ ]:
hcat(1.0)

In [ ]:
function